In [ ]:
!pip install google-search-results

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for google-search-results: filename=google_search_results-2.4.1-py3-none-any.whl size=25788 sha256=4d9b387cff9259d00c4fa837ab7b561929dddf5e1d81ef40776ecb1f998777d3
  Stored in directory: /root/.cache/pip/wheels/82/a3/c5/364155118f298722dff2f79ae4dd7c91e92b433ad36d6f7e0e
Successfully built google-search-results


In [ ]:
import os, shutil, requests, lxml, re, json, urllib.request
from bs4 import BeautifulSoup
from serpapi import GoogleSearch
import socket
import hashlib

socket.setdefaulttimeout(15)

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.114 Safari/537.36"
}

def google_images_search(text, dst, qt=None, page=0, remain=0):
    params = {
        "q": text,        # search query
        "tbm": "isch",  # image results
        "hl": "pt",     # language of the search
        "gl": "br",     # country where search comes from
        "ijn": str(page)      # page number
    }

    param_soup = BeautifulSoup(requests.get("https://www.google.com.br/search", params=params, headers=headers, timeout=30).text, "lxml")

    all_script_tags = param_soup.select("script")

    matched_images_data = "".join(re.findall(r"AF_initDataCallback\(([^<]+)\);", str(all_script_tags)))
    matched_images_data_fix = json.dumps(matched_images_data)
    matched_images_data_json = json.loads(matched_images_data_fix)
    matched_google_image_data = re.findall(r'\"b-GRID_STATE0\"(.*)sideChannel:\s?{}}', matched_images_data_json)
    matched_google_images_thumbnails = ", ".join(re.findall(r'\[\"(https\:\/\/encrypted-tbn0\.gstatic\.com\/images\?.*?)\",\d+,\d+\]', str(matched_google_image_data))).split(", ")
    thumbnails = [bytes(bytes(thumbnail, "ascii").decode("unicode-escape"), "ascii").decode("unicode-escape") for thumbnail in matched_google_images_thumbnails]
    removed_matched_google_images_thumbnails = re.sub(r'\[\"(https\:\/\/encrypted-tbn0\.gstatic\.com\/images\?.*?)\",\d+,\d+\]', "", str(matched_google_image_data))
    matched_google_full_resolution_images = re.findall(r"(?:'|,),\[\"(https:|http.*?)\",\d+,\d+\]", removed_matched_google_images_thumbnails)

    full_res_images = [bytes(bytes(img, "ascii").decode("unicode-escape"), "ascii").decode("unicode-escape") for img in matched_google_full_resolution_images]

    if not os.path.exists("Images/"):
      print("Creating new images folder")
      os.mkdir("Images/")
    
    for index, (metadata, thumbnail, original) in enumerate(zip(param_soup.select('.isv-r.PNCib.MSM1fd.BUooTd'), thumbnails, full_res_images), start=len(dst)):
        imgtitle = metadata.select_one(".VFACy.kGQAp.sMi44c.lNHeqe.WGvvNb")["title"]

        try:
          opener=urllib.request.build_opener()
          opener.addheaders=[('User-Agent','Mozilla/5.0 (Windows NT 10.0; Win64; x64)')]
          urllib.request.install_opener(opener)

          fname = f'Images/{index}.jpg'

          urllib.request.urlretrieve(original, fname)

          with open(fname, "rb") as f:
            file_hash = hashlib.blake2b()
            chunk = f.read(8192)
            while chunk:
                file_hash.update(chunk)
                chunk = f.read(8192)

            for img in dst:
              if img["title"] == imgtitle or img["hash"] == file_hash.hexdigest():
                os.remove(fname)
                raise Exception("Warning: Duplicate file detected")

          dst.append({
              "title": imgtitle,
              "tag": text,
              #"link": metadata.select_one(".VFACy.kGQAp.sMi44c.lNHeqe.WGvvNb")["href"],
              #"source": metadata.select_one(".fxgdke").text,
              #"thumbnail": thumbnail,
              #"original": original,
              "hash": file_hash.hexdigest()
          })

          print(f"Image file quantity: {len(os.listdir('Images/'))}")

        except Exception as e: print(e)

        if len(dst) == qt+remain and qt is not None:
          break
      
    print(f"------------ query: {text} | image list size: {len(dst)} | page: {page} ------------")

    if not os.path.exists("tmp_img/"):
      os.mkdir("tmp_img/")

    files = sorted(os.listdir("Images/"))[remain:]

    for i, file in enumerate(files, start=remain):
      shutil.copyfile(f'Images/{file}', f'tmp_img/{i}.jpg')

    shutil.rmtree('Images')
    os.rename('tmp_img', 'Images')

    if len(dst) < qt+remain and qt is not None:
      google_images_search(text, dst, qt=qt, page=(page + 1), remain=remain)

In [ ]:
!rm -rf Images
!rm -rf tmp_img

In [ ]:
gimages = []
google_images_search("camisa", gimages, qt=50) #Inicia a função asssim
google_images_search("bermuda", gimages, qt=50, remain=len(gimages)) #Para cada tag nova, adiciona mais uma chamada no modo recursivo
google_images_search("regata", gimages, qt=50, remain=len(gimages)) #Para cada tag nova, adiciona mais uma chamada no modo recursivo

#Recomendo por em um loop com a lista das tags, mas não tive tempo de testar ainda.

Creating new images folder
Image file quantity: 1
Image file quantity: 2
Image file quantity: 3
Image file quantity: 4
Image file quantity: 5
Image file quantity: 6
Image file quantity: 7
Image file quantity: 8
Image file quantity: 9
Image file quantity: 10
Image file quantity: 11
Image file quantity: 12
Image file quantity: 13
Image file quantity: 14
Image file quantity: 15
Image file quantity: 16
Image file quantity: 17
Image file quantity: 18
Image file quantity: 19
Image file quantity: 20
Image file quantity: 21
Image file quantity: 22
Image file quantity: 23
Image file quantity: 24
Image file quantity: 25
Image file quantity: 26
Image file quantity: 27
Image file quantity: 28
Image file quantity: 29
Image file quantity: 30
Image file quantity: 31
Image file quantity: 32
Image file quantity: 33
Image file quantity: 34
Image file quantity: 35
Image file quantity: 36
Image file quantity: 37
Image file quantity: 38
Image file quantity: 39
Image file quantity: 40
Image file quantity: 4

In [ ]:
import pandas as pd

df = pd.DataFrame(gimages)
df

,title,tag,hash
0,Oferta de Camisa Nike Brasil I 2022/23 Jogador...,camisa,325ce2742410c4e524309ac9567f14ea2fc6c2428bbbf9...
1,Camisa de Sarja Endurance - Preta | Invictus,camisa,9fdd04717dc4c867218aa8012aa5d274b8b09ec6bb9ba3...
2,Camisa Manga Curta Slim em Algodão Fio a Fio -...,camisa,a651401f1bb22e16472c7a379e86e9d4063d144f428a75...
3,Camisa social branca masculina de microfibra m...,camisa,20d5256506faa1e05582f2902f21ae3f97f738ee59f3fb...
4,Camisa Social Masculina Slim Manga Longa Azul ...,camisa,9e26734a8db81c66120bf2d21437c4e3a68f35e190c779...
...,...,...,...
145,REGATA MASCULINA MIZUNO,regata,7322428dd86f79e1c6453e7956e32c9829a4c6801b3673...
146,Regata Branca Feminina Adulto | Ribana Poliést...,regata,57a58d5397133bf5595dff294621c4dc6e61e39f8a8fab...
147,Regata TFM Poliviscose | Branca - Elite,regata,0bbce1938f9fc353da1e173960824c9e144098dbfab721...
148,Regata Algodão Branca - Compre Já - UW - Casa ...,regata,c9b7db7ef2d0197ac77b0d31715f4f51e362469937d9d2...
